# Imports

In [ ]:
# Move to the root
import os
cwd = os.getcwd()
if os.path.basename(cwd) != "cv-in-farming":
    os.chdir("../")
print(os.getcwd())

In [ ]:
import random

import matplotlib.pyplot as plt
from matplotlib import colors

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
from utils.helpers import create_template
from src.dataloader import FurrowDataset
from src.image_processing import apply_template_matching

# Load Frames

In [ ]:
# Input: Enter folder holding frames on which Template Matching will be applied.
folder = "dataset/train/20201112_125754"

dataset_args = {
  "data_path": folder,
  "load_edge": True,
  "load_time": False,
}
dataset = FurrowDataset(dataset_args)
print(dataset)

# Apply Template Matching

In [ ]:
# Input: Output path to store edge coordinates and plots. If None, results are visualized on the notebook only.
output_path = None

# Configuration of Template Matching when camera is mounted to the front.
front_config = {
    "out": output_path,
    "template_size": 30,
    "position": 1,
    "start_depth": 0.92, # Given in depth-scale
    "contour_width": 25, # Given in y-scale
    "y_step": 5,         # Given in y-scale
    "n_contours": 1000,
    "ransac_thresh": 30,
    "score_thresh": None,
    "roi": [None,None,250,None], # min_y:max_y, min_x:max_x
    "fit_type": "curve",
    "verbose": 0
}

# Configuration of Template Matching when camera is mounted to the back.
back_config = {
    "out": output_path,
    "template_size": 30,
    "position": 2,
    "start_depth": 1.10, # Given in depth-scale
    "contour_width": 25, # Given in y-scale
    "y_step": 5,         # Given in y-scale
    "n_contours": 1000,
    "ransac_thresh": 10,
    "score_thresh": None,
    "roi": [None,None,250,450], # min_y:max_y, min_x:max_x
    "fit_type": "curve",
    "verbose": 0
}

In [ ]:
# TODO: Saving Matplotlib plots causes memory leak over time. To save plots, a different module should be used.
# Matplotlib used to obtain scatter plots. Any alternative library for this purpose can be used.
import gc

# Given set of inliers, set of outliers, pixel coordinates for edges, prepare a plot overlaying these on depth array.
def prepare_corner_plot(depth_arr, inliers=None, outliers=None, edge_pixels=None):
    plt.figure(figsize=(10,10))
    depth_arr = np.rint(255 * (depth_arr / depth_arr.max()))
    depth_arr = np.clip(depth_arr * 7, a_min=0, a_max=255).astype(np.uint8)
    plt.imshow(depth_arr, cmap="gray")
    if edge_pixels is not None:
        plt.plot(edge_pixels[:,1], edge_pixels[:,0], color="springgreen", linewidth=2)
    if inliers is not None:
        inlier_pts = plt.scatter(inliers[:,1], inliers[:,0], color="cyan", marker="o")
    if outliers is not None:
        outlier_pts = plt.scatter(outliers[:,1], outliers[:,0], color="red", marker="x")
    if inliers is not None and outliers is not None:
        plt.legend((inlier_pts, outlier_pts), ("inliers", "outliers"), loc=1)

# Given pixel coordinates for edges, prepare a plot overlaying edge on RGB image.
def prepare_overlay_plot(image, edge_pixels, cstr="springgreen"):
    plt.figure(figsize=(10,10))
    plt.imshow(image)
    plt.plot(edge_pixels[:,1], edge_pixels[:,0], color=cstr, linewidth=2)

# Run Template Matching for a frame based on config, store the results to specified folder.
def execute(frame_idx, depth_arr, rgb_img, 
            out,
            template_size,
            position,
            start_depth,
            contour_width,
            y_step,
            n_contours,
            ransac_thresh,
            score_thresh,
            roi,
            fit_type,
            verbose):
    
    # Create a template to find corners
    template = create_template(size=template_size, position=position)
    
    # Fit a curve (2nd degree polynomial) to inlier detections
    edge_pixels, inliers, outliers = apply_template_matching(depth_arr,
                                         template,
                                         start_depth=start_depth,     # Given in depth-scale
                                         contour_width=contour_width, # Given in y-scale
                                         y_step=y_step,               # Given in y-scale
                                         n_contours=n_contours,
                                         ransac_thresh=ransac_thresh,
                                         score_thresh=score_thresh,
                                         roi=roi,
                                         fit_type=fit_type,
                                         verbose=verbose)
    
    # Store pixel coordinates for the edge
    if out is not None:
        np.save(f"{out}/{frame_idx}_edge_pts.npy", edge_pixels)
      
    # Visualize or store inlier and outlier corners and fitted curve
    prepare_corner_plot(depth_arr, inliers, outliers, edge_pixels)
    if out is None:
        plt.show()
    else:
        plt.savefig(f"{out}/{frame_idx}_edge_vis.png")
        # Clear the current axes.
        plt.cla() 
        # Clear the current figure.
        plt.clf() 
        # Closes all the figure windows.
        plt.close('all')
        gc.collect()

    # Visualize or store the mask overlay on original RGB
    prepare_overlay_plot(rgb_img, edge_pixels)
    if out is None:
        plt.show()
    else:
        plt.savefig(f"{out}/rgb_overlay_{frame_idx}.png")
        # Clear the current axes.
        plt.cla() 
        # Clear the current figure.
        plt.clf() 
        # Closes all the figure windows.
        plt.close('all')
        gc.collect()

## Execute on all frames available

In [ ]:
load_darr = True
load_rgb = True
load_drgb = False
load_edge = False
load_time = False
num_frames = len(dataset)

for i in range(num_frames):
    item = dataset.get_frame_files(i, 
                                   load_darr=load_darr,
                                   load_rgb=load_rgb,
                                   load_drgb=load_drgb,
                                   load_edge=load_edge,
                                   load_time=load_time)
    frame_idx = item['frame_id']
    rgb_img = np.array(item['rgb_img'])
    depth_arr = item['depth_arr']
    
    print(f"Detection on frame-{frame_idx}:")
    execute(frame_idx, depth_arr, rgb_img, **front_config)
    print()

## Pick one frame and execute

In [ ]:
# Input: Use this cell to pick a desired frame or a frame at random.

# Pick a frame at random:
# num_frames = len(dataset)
# file_indices = list(range(num_frames))
# i = random.choice(file_indices)
# print("Frame choice:", i)

# Pick a desired frame:
i = 1287

In [ ]:
load_darr = True
load_rgb = True
load_drgb = False
load_edge = False
load_time = False
item = dataset.get_frame_files(i, 
                               load_darr=load_darr,
                               load_rgb=load_rgb,
                               load_drgb=load_drgb,
                               load_edge=load_edge,
                               load_time=load_time)
frame_idx = item['frame_id']
rgb_img = np.array(item['rgb_img'])
depth_arr = item['depth_arr']

In [ ]:
# Show depth array (brightness adjusted)
plt.figure(figsize=(10,10))
depth_arr = np.rint(255 * (depth_arr / depth_arr.max()))
adjusted = np.clip(depth_arr * 7, a_min=0, a_max=255).astype(np.uint8)
plt.imshow(adjusted, cmap="gray")

In [ ]:
# Show cropped depth array (brightness adjusted)
plt.figure(figsize=(10,10))
adjusted = depth_arr[:,250:]
adjusted = np.rint(255 * (adjusted / depth_arr.max()))
adjusted = np.clip(adjusted * 7, a_min=0, a_max=255).astype(np.uint8)
plt.imshow(adjusted, cmap="gray")

In [ ]:
print(f"Detection on frame-{frame_idx}:")
execute(frame_idx, depth_arr, rgb_img, **front_config)

# Generate Video from Existing Detections

* Ready edge masks are used for overlaying. Assumes that edge masks are previously stored.

In [ ]:
from matplotlib.animation import FFMpegWriter

plt.rcParams['animation.ffmpeg_path'] = '/usr/bin/ffmpeg'

# Input: Folder containing RGB frames and pixel coordinates for detected edges on them.
folder = "dataset/train/20201112_125754"
# Input: Frame index range to include in the output video (a slice of 30 frames corr. to 1 sec in 30 FPS)
frame_slice = slice(150,300)
# Input: Name for the output video.
output_name = "Template Matching Demo X"

dataset_args = {
  "data_path": folder,
  "load_edge": True,
  "load_time": False,
}
dataset = FurrowDataset(dataset_args)
print(dataset)

fig = plt.figure()

num_frames = len(dataset)
video_cut = list(range(num_frames))[frame_slice]

metadata = dict(title=f"{output_name}")
writer = FFMpegWriter(fps=30, metadata=metadata)

fig = plt.figure()
imgh = plt.imshow(np.zeros((480, 640), dtype=np.uint8))
ph, = plt.plot([], [], color="cyan", linewidth=2)

load_darr = False
load_rgb = True
load_drgb = False
load_edge = True
load_time = False

with writer.saving(fig, f"{output_name}.mp4", 100):
    for i in video_cut:
        item = dataset.get_frame_files(i, 
                                       load_darr=load_darr,
                                       load_rgb=load_rgb,
                                       load_drgb=load_drgb,
                                       load_edge=load_edge,
                                       load_time=load_time)
        frame_idx = item['frame_id']
        rgb_img = np.array(item['rgb_img'])
        edge_pixels = item['edge_pixels']
        
        # Replace plot with new frame
        imgh.set_data(rgb_img)
        # Draw respective edge
        ph.set_data(edge_pixels[:,1], edge_pixels[:,0])
        
        writer.grab_frame()